## Imports

In [ ]:
import scipy.signal

import numpy as np
from scipy.signal import welch
import os
import re
import pandas as pd
import librosa
import csv
from scipy.signal import resample_poly
from typing import Union, List, Dict, Tuple, Any

## Feature Extractor function
It works on a single 9s segment of 48kHz and returns a dictionary of features.

In [ ]:
def compute_lpc_apnea_features(
    audio: np.ndarray,
    sr: int = 16000,
    frame_length_ms: float = 25.0,
    hop_length_ms: float = 10.0,
    lpc_order: int = 12,
    n_fft: int = 512,
    delta_widths = (0, 3, 4, 5),
) -> Dict[str, Any]:
    """
    Compute LPC-based spectral-envelope features (and derivative/std summaries)
    intended for sleep-apnea-from-sound experiments.

    Parameters
    ----------
    audio : np.ndarray
        1D audio array (mono). Example: 9 seconds at 16 kHz.
    sr : int
        Sample rate (default 16000).
    frame_length_ms : float
        Frame length in milliseconds for short-time framing (default 25 ms).
    hop_length_ms : float
        Hop length in milliseconds (default 10 ms).
    lpc_order : int
        LPC order for each frame (default 12).
    n_fft : int
        Number of frequency bins to evaluate spectral envelope (default 512).
    delta_widths : iterable
        Iterable of integer delta widths to compute temporal deltas of the envelope.
        We use 0 to indicate "no delta" (original envelope), others are passed
        to librosa.feature.delta as `width=width*2+1` internally.

    Returns
    -------
    features : dict
        Dictionary with summary features. Keys include:
          - 'env_mean' : mean over all frames and freqs of spectral envelope
          - 'env_std'  : std over all frames and freqs of spectral envelope
          - 'freq_deriv_mean' : mean of frequency-derivative (env diff along freq)
          - 'freq_deriv_std'  : std of frequency-derivative
          - 'delta_{w}_std'   : overall std of temporal delta of envelope for width w
          - 'delta_{w}_mean'  : overall mean of temporal delta of envelope for width w
        and also per-frequency / per-frame arrays are returned under keys if needed.
    """
    # --- frame/hop sizes in samples
    frame_len = int(round(sr * (frame_length_ms / 1000.0)))
    hop_len = int(round(sr * (hop_length_ms / 1000.0)))
    if frame_len <= lpc_order:
        raise ValueError("frame length (samples) must be > lpc_order")

    # Short-time framing (librosa.util.frame helps but we'll use simple framing approach)
    # Use librosa.util.frame to create a 2D array (frames x frame_len)
    frames = librosa.util.frame(audio, frame_length=frame_len, hop_length=hop_len).T  # shape (n_frames, frame_len)

    n_frames = frames.shape[0]

    # Frequency grid for spectral envelope
    w, _ = scipy.signal.freqz([1.0], [1.0], worN=n_fft, fs=sr)  # returns freqs and response for placeholders
    freqs = w[:n_fft] if len(w) >= n_fft else w

    # Container for envelopes: shape (n_frames, n_fft)
    envelopes = np.zeros((n_frames, n_fft), dtype=float)

    for i in range(n_frames):
        frame = frames[i] * np.hamming(frame_len)  # window
        # compute LPC (librosa.lpc returns coefficients a: [1, -a1, -a2, ...] style)
        try:
            a = librosa.lpc(frame, order=lpc_order)
        except np.linalg.LinAlgError:
            # if LPC fails due to ill-conditioned autocorrelation, fallback to zeros
            a = np.zeros(lpc_order + 1, dtype=float)
            a[0] = 1.0
        # Compute frequency response of the all-pole filter 1/A(z) -> magnitude is spectral envelope
        # scipy.signal.freqz expects numerator (b) and denominator (a). For LPC, numerator is 1.
        freq_axis, h = scipy.signal.freqz([1.0], a, worN=n_fft, fs=sr)
        mag = np.abs(h)
        # Avoid zeros and stabilize with small epsilon
        envelopes[i, :] = mag + 1e-12

    # Global envelope statistics
    env_mean = float(np.mean(envelopes))
    env_std = float(np.std(envelopes))

    # Frequency derivative (first diff along frequency axis)
    freq_deriv = np.diff(envelopes, axis=1)  # shape (n_frames, n_fft-1)
    freq_deriv_mean = float(np.mean(freq_deriv))
    freq_deriv_std  = float(np.std(freq_deriv))

    # Temporal deltas of the envelope:
    # librosa.feature.delta uses width = 2 * N + 1 where N is the order.
    # We'll map delta_width w -> librosa width = 2*w + 1 (so w=3 -> width 7)
    delta_stats = {}
    for w in delta_widths:
        if w == 0:
            # interpret width 0 as the original envelope (no delta)
            delta_env = envelopes.copy()
        else:
            # compute delta along time axis for each frequency bin
            # librosa expects shape (n_freq, n_frames) -> so transpose
            delta_env = librosa.feature.delta(envelopes.T, width=(2*w + 1), order=1, axis=1).T
            # librosa.feature.delta returns same shape as input
        # overall summary
        key_mean = f"delta_{w}_mean"
        key_std  = f"delta_{w}_std"
        delta_stats[key_mean] = float(np.mean(delta_env))
        delta_stats[key_std]  = float(np.std(delta_env))

    # Pack features
    features = {
        "env_mean": env_mean,
        "env_std": env_std,
        "freq_deriv_mean": freq_deriv_mean,
        "freq_deriv_std": freq_deriv_std,
    }
    features.update(delta_stats)

    # # Optionally include some arrays for deeper inspection (but keep them optional)
    # features["_debug"] = {
    #     "n_frames": n_frames,
    #     "frame_len": frame_len,
    #     "hop_len": hop_len,
    #     "lpc_order": lpc_order,
    #     "n_fft": n_fft,
    #     # Note: don't include entire envelopes if very large; included here for completeness
    #     # Comment out if you want only summary stats:
    #     # "envelopes": envelopes,
    #     # "freq_deriv": freq_deriv,
    # }

    return features

## The main iterator function
For rapid parallel processing of all patients at the same time, Jess has created this framework that gets one patient ID at a time from the SBATCH. It is obtained by `os.environ.get()` and the patient ID is supplied by the `.sh` and `.sbatch` files combined. 

## Testing steps
1. Before doing the final sbatch, we run this test notebook. Since we won't be getting parallel patient IDs from sbatch, we will manually set one patient ID and the input output directories (and comment out the original `os.environ.get()` block, as you can see in the below code).
2. Call your custom feature function in the `#custom feature function goes here` part. Ensure the right indentation of the code.
3. Your feature function is expected to return a dictionary of features (with feature names as keys). After we get the features in the `feats` variable, we add the labels to the dictionary. I also added the segment number to preserve sequence order information, in case we want to use it in RNN or LSTM.
4. The rest of the code will write the features in a CSV file. The column names will be the feature names and "label" and "segment_index". The output file name will be the `patient_ID_feature_name.csv`.
5. **MAKE SURE** to adjust the `output_file` variable in the `main()` function and adjust the feature_name part in the output csv file name.
6. While testing your code in this notebook before the final SBATCH, **MAKE SURE** to adjust the `output_dir` variable in the `main()` function. We don't want to overwrite any other previously extracted features.
7. The output file of this code will be a single csv file for a patient. The number of rows in this csv file will be equal to the number of segments for that patient in the big `.npy` file. Each row will contain the features extracted for a segment and the corresponding label.
8. Once we see that the feature extraction function is working for a patient, we can move to sbatch.

## SBATCH steps after you are sure that the code works ok
1. We first need to prepare the python file. Currently, we have a ipynb notebook, which cannot be used with SBATCH. We need a `.py` file.
    1. Make sure your python code does not plot anything. Even if it does, save the plot to a file in an output directory. We don't need plots for features, we can do it later. So, try to skip any plotting.
    2. Copy all the codes in the cells of your notebook in a `.py` file.
    3. When you were testing the notebook, you commented out the `os.environ.get()` code block with test variables. Make sure to uncomment this `os.environ.get()` code block and delete/comment out the test variables part where you assigned values for testing on a single patient. 
3. Go to `job_full_dataset.sh`
    1. Adjust the `DATA_DIR`, `LABEL_DIR`, `OUTPUT_DIR`, `PATIENT_LIST`, `PYTHON_SCRIPT` variables.
    2. Keep in mind that when you are using a absolute path, it starts with a slash `/`. But when you are copying paths of a folder by right clicking on the folder in the file explorer inside Jupyterlab, that starting slash `/` is not included. You will have to adjust it.
4. After you are done with the `.sh` file, go to the `.sbatch` file.
    1. Change the following lines
        ```
        
        #SBATCH -c 32
        #SBATCH --mem=64G
        #SBATCH -t 0-01:00:00
        #SBATCH --array=0-70   # Must exactly match number of patients

        module load mamba/latest
        source activate tfenv1

        bash /scratch/sshuvo13/scratch_run_dir_sajib/LPC_sajib/job_full_dataset.sh
        
        ```
        1. `-c 32` is the number of cores of your CPU. `--mem=64G` is how many GBs of RAMS. `-t 0-01:00:00` is in the format of `D-HH-MM-SS` format. For example, if you want the time cap for your code to be 1 day, 2 hours, 3 minutes, and 1 seconds, then you set `-t 1-02:03:04`. Your code might be finished before this time, and this is the hard deadline. If you code does not finish before this, SOL will terminate your session. So, it is better to set up a bit more time than you think you might need in the `-t` variable. The `--array` value should be set as the number of patients in the `DATA_DIR`. For example, if there are 73 male patients, `--array=0-72`.
        2. Then you load mamba/latest and activate your environment as usual.
        3. Then comes the most important part. You call the `.sh` file you created earlier using `bash`. Make sure that if you are using absolute path, your path starts with the slash `/`.
5. Once you have your `.sbatch`, `.sh`, and `.py` files ready, recheck everything. Ensure that you set every paths correctly.
6. **MAKE SURE THAT YOU SAVED ALL THREE FILES**.
7. Then open a shell terminal within Jupyterlab by clicking on the plus (+) icon on top of the file browser, and scrolling all the way to the bottom.
8. Run the sbatch file using:
   ```
   sbatch your_sbatch_file_name.sbatch
   ```
9. You can monitor your sbatch by running:
    ```
    squeue -u $USER
    ```
10. If you want to cancel a sbatch job, note the SBATCH job id (a big number) and run:
    ```
    scancel your_sbatch_job_ID
    ```
11. If you want to see something similar to the task manager of Windows, run `top` in the shell, and then press `shift + m` to sort the list. Press `q` to quit the `top` environment.
12. If something is wrong in the `sbatch`, `sh`, or `py`, you will be able to see it in the `.err` files in the `./Logs/` directory, which will be in the same directory as your `.sbatch` file.
13. If you have used `print()` function in your `.py` code, you will see the print outputs in the `.out` files in the `./Logs/` directory. Usually, the `.out` files get populated at the end of code execution, rather than being continuously populated as the code runs. 

14. After your `.sbatch` run is complete, you will find all the patient's feature `.csv` files in the output directory. 

In [ ]:
def main(patient_id, data_dir, label_dir, output_dir):


    os.makedirs(output_dir, exist_ok=True)

    filename = f"{patient_id}_segmented.npy"
    segments_file_path = os.path.join(data_dir, filename)
    print(f"Opening (memmap) {segments_file_path}")
    segments = np.load(segments_file_path, mmap_mode='r')  # <-- memory-mapped

    if segments.ndim != 2:
        print(f"File {filename} does not contain a 2D array; skipping.")
        return

    # load labels (still likely small)
    label_file_name = f"{patient_id}_segments_labels.npy"
    label_file_path = os.path.join(label_dir, label_file_name)
    label_file = np.load(label_file_path)  # if this is huge, memmap it too

    number_of_segments = segments.shape[0]
    labels_subset = label_file[:number_of_segments]

    # Prepare CSV output (write header)
    output_file = os.path.join(output_dir, f"{patient_id}_LPC.csv")
    header_written = False

    # iterate by index to avoid copying the whole 'segments' at once
    for i in range(number_of_segments):
        try:
            seg = segments[i]            # this is a memmap slice (view)
        except Exception as e:
            print(f"Error reading segment {i}: {e}")
            continue

        # resample to 16k
        try:
            signal_16khz = resample_to_16k(seg, orig_sr=48000, target_sr=16000)
        except Exception:
            # fallback to librosa if needed
            signal_16khz = librosa.resample(np.asarray(seg, dtype=np.float32), orig_sr=48000, target_sr=16000)


        #custom feature function goes here
        feats = compute_lpc_apnea_features(audio = signal_16khz)
        # feats = {'Q_MFCC_avg': feats_raw}
        feats['label'] = labels_subset[i] if i < len(labels_subset) else None
        feats['segment_index'] = i

        # write row immediately to CSV (append)
        if not header_written:
            with open(output_file, 'w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=list(feats.keys()))
                writer.writeheader()
                writer.writerow(feats)
            header_written = True
        else:
            with open(output_file, 'a', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=list(feats.keys()))
                writer.writerow(feats)

    print(f"Finished {filename}, wrote {output_file}")
    

if __name__ == "__main__":
    # Read environment variables set by job.sh
    # patient_id = os.environ.get("PATIENT_ID")
    # data_dir = os.environ.get("DATA_DIR")
    # label_dir = os.environ.get("LABEL_DIR")
    # output_dir = os.environ.get("OUTPUT_DIR")

    patient_id = '00000995-100507'
    data_dir = "/scratch/sshuvo13/project_shared_folder_bspml_1/segmented_edfs/female_segmented_edfs"
    label_dir="/scratch/sshuvo13/project_shared_folder_bspml_1/rml_analysis/segment_csv_data/labels_of_each_segment"
    output_dir="/scratch/sshuvo13/project_shared_folder_bspml_1/whole_dataset_features/female/LPC_sajib"

    # Simple check
    if not all([patient_id, data_dir, label_dir, output_dir]):
        raise ValueError("Missing required environment variables: PATIENT_ID, DATA_DIR, LABEL_DIR, OUTPUT_DIR")

    # print(f"\nProcessing patient: {patient_id}")


    

    main(patient_id, data_dir, label_dir, output_dir)